## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
# %pip install pyspark

In [0]:

%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Delta Lake operations
from delta.tables import *

# Streaming data handling
from pyspark.streaming import StreamingContext

# MLflow for experiment tracking
import mlflow

# Plotly for visualizations
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Additional libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os

# Scikit-learn metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Time library
import time


In [0]:
# Ensure shuffle partitions are optimized for the cluster size; may need to adjust based on actual cluster config
spark.conf.set("spark.sql.shuffle.partitions", "4")  # Adjust based on your cluster's core count and performance tuning

print("Libraries imported successfully, and shuffle partitions set.")

# Reminder: Ensure that your runtime version matches the requirements of the libraries used, especially for Delta and MLflow.

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Step 2.0: Explore and understand the source data

# Use Databricks' dbutils to list source files directly

source_files = get_source_listing_df()


file_count = source_files.count()

print(f"Number of source files: {file_count}")





In [0]:
# Count the number of source files and print the count
# file_count = source_files.count()
# print(f"Number of source files: {file_count}")

In [0]:
source_files.head()

In [0]:
display(show_s3_file_contents("voc_volume/0.json"))

In [0]:
# Displaying contents of the file 'voc_volume/0.json' after decoding it to utf-8
file_content = show_s3_file_contents("voc_volume/0.json").decode('utf-8')
print("Contents of the file 'voc_volume/0.json':")
print(file_content)


## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Note_Brynn: REMOVE this block after testing with small sample.
# List files in the temporary directory to verify
#display(dbutils.fs.ls(temp_dir))

In [0]:
# Define the schema based on data provided
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True)
])

In [0]:
# Setup a read stream with cloudFiles
raw_stream = (
    spark.readStream
    .format("cloudFiles")            # Utilize cloudFiles for cloud-based sources
    .option("cloudFiles.format", "json")     # Specify the file format within cloudFiles
    #.option("maxFilesPerTrigger", 1)
    .schema(raw_schema)                      # Enforce the schema defined above
    .load(TWEET_SOURCE_PATH)               # Load data from the defined path
    .withColumn("source_file", input_file_name())  # Capture the source file name
    .withColumn("processing_time", current_timestamp())  # Add a processing timestamp
)


In [0]:
# raw to bronze stream
# Setup a write stream to append to the bronze delta table
bronze_stream = (
    raw_stream.writeStream
    .outputMode("append")
    .format("delta")
    .option("mergeSchema", "true")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .queryName("bronze_stream")
    .trigger(once=True)  # Process all available data at once and then stop
    .start(BRONZE_DELTA)
)

# Await termination and handle stream
bronze_stream.awaitTermination()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:

bronze_table = spark.read.format("delta").load(BRONZE_DELTA)
display(bronze_table)

In [0]:
# Count the total number of tweets
tweet_count = bronze_table.count()
print("The number of tweets captured in the Bronze Table: ", tweet_count)


In [0]:

# Check for null values in each column
null_counts = bronze_table.select([count(when(col(c).isNull(), c)).alias(c) for c in bronze_table.columns])
print("Null values in each column:")
null_counts.show()


Plan to handle nulls:
Filture out tweets with null. 

For example, assuming 'text' and 'user' fields are critical and should not be null:

filtered_bronze_table = bronze_table.dropna(subset=["text", "user"])
print("Filtered out tweets with null 'text' or 'user'.")

In [0]:
# Count the number of tweets by each unique user handle and sort the data by descending count
user_tweet_counts = bronze_table.groupBy("user").count().orderBy("count", ascending=False)
print("Top tweeters:")
user_tweet_counts.show()

In [0]:
# Count how many tweets have at least one mention (@)
tweets_with_mentions = bronze_table.filter(col("text").contains("@")).count()
tweets_without_mentions = tweet_count - tweets_with_mentions
print(f"Tweets with mentions: {tweets_with_mentions}")
print(f"Tweets without mentions: {tweets_without_mentions}")

In [0]:
top_tweeters = user_tweet_counts.limit(20).toPandas()
fig = px.bar(top_tweeters, x='user', y='count', title="Top 20 Tweeters")
fig.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
bronze_table.dtypes

In [0]:
# Note_Brynn: RECOVER this cell after testing with small sample.
# Read from the Bronze Delta table as a stream

bronze_df = spark.readStream.format("delta").load(BRONZE_DELTA)

#bronze_table = spark.read.format("delta").load(BRONZE_DELTA)

In [0]:
# Transform the Bronze Data to Silver Data
silver_transform = bronze_df \
    .withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss zzz yyyy")) \
    .withColumn("mentions", explode(split(col("text"), " "))) \
    .filter(col("mentions").startswith("@")) \
    .withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", "")) \
    .select(
        col("timestamp"),
        col("mentions").alias("mention"),
        col("cleaned_text"),
        col("sentiment")
    )


In [0]:
silver_transform.dtypes

In [0]:
silver_stream = (
    silver_transform.writeStream
    .outputMode("append")
    .format("delta")
    #.option("mergeSchema", "true")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .queryName("silver_stream")
    .trigger(once=True)
    .start(SILVER_DELTA)
)


# Await termination and handle stream
silver_stream.awaitTermination()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
#   mode="complete", ?????

In [0]:
# Set up the model URI for the pre-trained sentiment analysis model
model_uri = f"models:/{MODEL_NAME}/Production"


In [0]:
# Load the model as a Spark UDF
sentiment_analysis_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri, result_type=StringType())


In [0]:
# Register the UDF
spark.udf.register("sentiment_analysis", sentiment_analysis_udf)


In [0]:
# Define the schema for the JSON structure returned by the UDF
prediction_schema = StructType([
    StructField("label", StringType(), True),
    StructField("score", DoubleType(), True)
])

In [0]:
# Note_Brynn: RECOVER this cell after testing with small sample.

# Read from the Silver Delta table as a stream
silver_df = spark.readStream.format("delta").load(SILVER_DELTA)


In [0]:
# Transform Silver data to Gold data
gold_transform = (
    silver_df
    .withColumn("predictions", from_json(sentiment_analysis_udf(col("cleaned_text")), prediction_schema))
    .withColumn("predicted_sentiment", col("predictions.label"))
    .withColumn("predicted_score", (col("predictions.score") * 100).cast(DoubleType()))
    .withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))
    .withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "positive", 1).otherwise(0))
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment"),
        col("predicted_score"),
        col("predicted_sentiment"),
        col("sentiment_id"),
        col("predicted_sentiment_id")
    )
)

In [0]:
gold_transform.dtypes

In [0]:
gold_stream = (
    gold_transform.writeStream
    .outputMode("append")
    .format("delta")
    .option("mergeSchema", "true")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .queryName("gold_stream")
    .start(GOLD_DELTA)
)



gold_stream.awaitTermination()


## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:

spark = SparkSession.builder.getOrCreate()

In [0]:
gold_df = spark.read.format("delta").load(GOLD_DELTA)


In [0]:
print(type(gold_df))

In [0]:
print(gold_df.head())

In [0]:
print(gold_df.dtypes)

In [0]:
# Using Spark MLlib to evaluate predictions
evaluator = MulticlassClassificationEvaluator(
    labelCol="sentiment_id", predictionCol="predicted_sentiment_id", metricName="f1")
f1 = evaluator.evaluate(gold_df)
precision = evaluator.evaluate(gold_df, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(gold_df, {evaluator.metricName: "weightedRecall"})


In [0]:
# Log metrics in MLflow
with mlflow.start_run():
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1-Score", f1)

    # Assuming you handle plotting and saving the confusion matrix elsewhere if needed

    # Log model and MLflow version as parameters
    mlflow.log_param("Model Name", MODEL_NAME)
    mlflow.log_param("MLflow Version", mlflow.__version__)

    # Handling Delta Table version
    silver_delta_path = SILVER_DELTA  
    silver_table = DeltaTable.forPath(spark, silver_delta_path)
    silver_version = silver_table.history(1).select("version").collect()[0][0]
    mlflow.log_param("Silver Table Version", silver_version)

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
while True:
    # Load the latest snapshot of the data
    gold_df = spark.read.format("delta").load(GOLD_DELTA)

    # Treat neutral as negative
    gold_df = gold_df.withColumn(
        "predicted_sentiment",
        F.when(F.col("predicted_sentiment") == "neutral", "negative").otherwise(F.col("predicted_sentiment"))
    )
    
    # Count total mentions
    total_mentions = gold_df.select("mention").distinct().count()

    # Aggregate sentiments counts for each mention
    mention_sentiment_counts = gold_df.groupBy("mention").agg(
        F.count(F.when(F.col("predicted_sentiment") == "positive", True)).alias("positive"),
        F.count(F.when(F.col("predicted_sentiment") == "negative", True)).alias("negative")
    ).withColumn("total", F.col("positive") + F.col("negative"))

    # Sort by total mentions count in descending order
    sorted_mentions = mention_sentiment_counts.orderBy(F.col("total").desc())

    # Get the top 20 mentions with positive sentiment
    top_positive_mentions = sorted_mentions.limit(20).select("mention", "positive").toPandas()
    fig_positive = px.bar(top_positive_mentions, x='mention', y='positive', title="Top 20 Positive Mentions")
    fig_positive.show()

    # Get the top 20 mentions with negative sentiment
    top_negative_mentions = sorted_mentions.limit(20).select("mention", "negative").toPandas()
    fig_negative = px.bar(top_negative_mentions, x='mention', y='negative', title="Top 20 Negative Mentions")
    fig_negative.show()

    # Optionally print the total number of mentions
    print("Total mentions in the dataset:", total_mentions)

    # Wait before the next update
    time.sleep(300)  # Sleep for 5 minutes


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

elapsed_time = END_TIME - START_TIME  # Calculate the total execution time
print(f"Total execution time: {elapsed_time:.2f} seconds")  


## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE